# Homework 4

In homework 2, we will work with log content of user in 30 days. The general idea here is we first try to analyze one day
then apply it to a month. Thanks to **pyspark**  supporting glob pattern directory, this can be done with ease. 


In [1]:
import ast
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import greatest
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, to_date, when, \
	input_file_name, regexp_extract, sum, lit
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, FloatType


In [3]:
import pandera

In [2]:
builder = (
	SparkSession.builder.appName("homework1")
	.config("spark.driver.memory", "16g")
	.config("spark.driver.cores", 4)
	.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
	.config(
		"spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"
	)
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/12/14 00:41:49 WARN Utils: Your hostname, Andrea-Le-MBP-Pro.local resolves to a loopback address: 127.0.2.2; using 192.168.1.4 instead (on interface en0)
23/12/14 00:41:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/sonle/.sdkman/candidates/spark/3.4.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sonle/.ivy2/cache
The jars for the packages stored in: /Users/sonle/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2cb331e-ce16-424c-b676-93f6f93e6d59;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 273ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Parquet is all the way
Log content here is stored on disk in **json format** which is not well-suited for reading purpose. 
Hence, we should load and write it down as **parquet format** since parquet is more toward to analytical workload. 

In [3]:
log_path = "/Users/sonle/Documents/GitHub/spark-playground/data/log_content"
data_path = "/Users/sonle/Documents/GitHub/spark-playground/data/"
parquet_path = "parquet_log_content.parquet"

In [ ]:
# Just run this function one time
def read_log_content(
		data_paths: str,
		parquet_path: str
):
	"""
    Read and rename log content data in json format and 
    write it to disk in parquet format with partitioning option
    :param data_paths: data directory
    :param parquet_path: parquet file name
    :return: None
    """
	return (
		spark.read.json(f"{data_paths}/log_content/*.json")
		.select(
			col("_index").alias("Index"),
			col("_type").alias("Type"),
			col("_id").alias("Id"),
			col("_score").alias("Score"),
			col("_source.*"),
			to_date(
				regexp_extract(input_file_name(), r"\d{8}", 0),
				"yyyyMMdd"
			).alias("Date")
		)
		.write.parquet(
			path=f"{data_paths}/{parquet_path}",
			mode="overwrite",
			partitionBy="Date",
			compression="zstd"
		)
	)


read_log_content(data_path, parquet_path)

In [8]:
from pyspark.sql.types import StructType

from pyspark.sql.types import StructField, LongType

schema = StructType([
	StructField('_id', StringType(), True),
	StructField('_index', StringType(), True),
	StructField('_score', LongType(), True),
	StructField('_source',
				StructType([StructField('AppName', StringType(), True),
							StructField('Contract', StringType(), True),
							StructField('Mac', StringType(), True),
							StructField('TotalDuration', LongType(), True)
							]
						   ), True
				),
	StructField('_type', StringType(), True)]
)

(

	spark.read.json(
		path=f"{data_path}/log_content/*.json", pathGlobFilter="*.json", schema=schema
	).select(
		col("_index").alias("Index"),
		col("_type").alias("Type"),
		col("_id").alias("Id"),
		col("_score").alias("Score"),
		col("_source.*"),
		to_date(regexp_extract(input_file_name(), r"\d{8}", 0), "yyyyMMdd").alias(
			"Date"
		),
	)
	.show()
)

+-------+-----+--------------------+-----+-------+---------+------------+-------------+----------+
|  Index| Type|                  Id|Score|AppName| Contract|         Mac|TotalDuration|      Date|
+-------+-----+--------------------+-----+-------+---------+------------+-------------+----------+
|history|kplus|AX_momhia1FFivsGrn9o|    0|  KPLUS|HNH579912|0C96E62FC55C|          254|2022-04-01|
|history|kplus|AX_momhca1FFivsGrnvg|    0|  KPLUS|HUFD40665|CCEDDC333614|         1457|2022-04-01|
|history|kplus|AX_momhaa1FFivsGrnny|    0|  KPLUS|HNH572635|B068E6A1C5F6|         2318|2022-04-01|
|history|kplus|AX_momhca1FFivsGrnvv|    0|  KPLUS|HND141717|08674EE8D2C2|         1452|2022-04-01|
|history|kplus|AX_momhia1FFivsGrn98|    0|  KPLUS|HNH743103|402343C25D7D|          251|2022-04-01|
|history|kplus|AX_momg9a1FFivsGrnkS|    0|  KPLUS|HNH893773|B84DEE76D3B8|          924|2022-04-01|
|history|kplus|AX_momhca1FFivsGrnwA|    0|  KPLUS|HND083642|B84DEE849A0F|         1444|2022-04-01|
|history|k

In [8]:
df = (
	spark.read.parquet(
		f"{data_path}/{parquet_path}"
	)
	.filter(col("Contract") != "0")
)


In [ ]:
import datetime

def generate_date_range(start_date, end_date, format="%Y-%m-%d"):
	"""Generates a range of dates in string format.
  
    Args:
      start_date: The start date of the range.
      end_date: The end date of the range.
      format: The format of the dates in the range.
  
    Returns:
      A list of dates in string format.
    """

	date_range = []
	current_date = start_date
	while current_date <= end_date:
		date_range.append(current_date.strftime(format))
		current_date += datetime.timedelta(days=1)
	return date_range

start_date = datetime.datetime(2022, 4, 1)
end_date = datetime.datetime(2022, 4, 3)
date_range = generate_date_range(start_date, end_date)
date_range

In [ ]:
list_file = [
	f"{data_path}{parquet_path}/Date={date}"
	for date in date_range
]
list_file

In [ ]:
(
	spark.read.parquet(
		*list_file
	)
	.filter((col("Contract") != "0") & (col("Contract") != ""))
	.explain("formatted")
)

In [ ]:
df.createOrReplaceTempView("log_content")
#generate date range


# ETL
The requirements at this part is that we need to extract the information of each contract in terms of total duration of five segments "TVDuration", "MovieDuration", "SportDuration",
"ChildDuration", "RelaxDuration".

The first option is to **aggregate sum function manually**, the second option is to use supported **pyspark pivot** method. 

Since we have 5 categories, we will need to rewrite some operations repeatedly which can take some time. For example:
```python
when(col("name") == "some_name", value).otherwise(value)
# or sum method in agg method
sum("columns").alias("columns_name")
```
Thankfully, pyspark functions support arguments unpacking `*exprs` so we can leverage this by creating lists of expressions before pasting them to pyspark functions.


In [10]:
 app_names = [
	"CHANNEL",
	"KPLUS",
	"VOD",
	"FIMS",
	"BHD",
	"SPORT",
	"CHILD",
	"RELAX",
]

column_names = [
	"TVDuration",
	"TVDuration",
	"MovieDuration",
	"MovieDuration",
	"MovieDuration",
	"SportDuration",
	"ChildDuration",
	"RelaxDuration",
]


In [ ]:
def summarize_by_manual_pivot(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str]
) -> DataFrame:
	"""
	Function to manually pivot data. 
	:param df: Dataframe, log content data
	:param app_names: application names in original data
	:param column_names: new column names after aggregation
	:return: new summarized data with required format
	"""

	whens = [when(col("AppName") == app_name, col("TotalDuration")).otherwise(lit(0)).alias(f"{column_name}")
			 for app_name, column_name in zip(app_names, column_names)
			 ]

	exprs = [sum(x).alias(f"{x}") for x in column_names]

	return (
		df
		.select(
			col("Contract"),
			*whens
		)
		.groupby("Contract")
		.agg(*exprs)
		.orderBy("TVDuration", ascending=False)
	)


In [ ]:
%%time
summarize_by_manual_pivot(df, app_names=app_map, column_names=columns_name)

## Pivot data

In [5]:
def pivot_data(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str],
) -> DataFrame:
	if len(app_names) != len(column_names):
		raise ValueError("The lengths of `app_names` and `column_names` must be the same.")
	
	whens = F
	for app_name, column_name in zip(app_names, column_names):
		whens = whens.when(col("AppName") == app_name, column_name)
	whens = whens.otherwise("Unknown").alias("Type")

	df = (
		df
		.select(
			col("Contract"),
			col("TotalDuration"),
			whens
		)
		.filter(
			(col("Contract") != "0") & (col("Type") != "Unknown") & (col("TotalDuration") > 0) 
		)
		.groupBy("Contract")
		.pivot("Type")
		.sum("TotalDuration")
	)

	return df

In [13]:
df = spark.read.format("delta").load("/Users/sonle/Documents/GitHub/spark-playground/data/delta/bronze")

In [14]:
pivot_df = pivot_data(df, app_names, column_names)


23/11/07 00:00:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
pivot_df.show()

+---------+-------------+-------------+-------------+-------------+----------+
| Contract|ChildDuration|MovieDuration|RelaxDuration|SportDuration|TVDuration|
+---------+-------------+-------------+-------------+-------------+----------+
|TVFD16104|         null|         null|         null|         null|   1135327|
|HYFD48206|         null|         null|         null|         null|    178706|
|NDFD14513|         null|         null|         null|         null|    960378|
|SLFD07508|         null|        32902|         null|         null|    593839|
|SGH227639|       134716|         null|         null|         null|    270153|
|HNJ070592|         null|         null|         null|         null|    141838|
|QND047956|         null|         null|         null|         null|   1430192|
|HNH297108|         null|         1102|          341|         null|    405295|
|TGD014492|         null|         null|         null|         null|    389646|
|HPH001741|         null|         null|         null

In [ ]:
most_watch = ["Child", "Movie", "Relax", "Sport", "TV"]
columns = df.columns[1:]
whens = F
largest_dur = F.greatest(*columns)
for column, name in zip(columns, most_watch):
	whens = whens.when(col(f"{column}") == largest_dur, lit(f"{column}"))
whens = whens.otherwise("Error").alias("MostWatch")
whens

In [ ]:
df.columns

In [ ]:
def most_watch(df: DataFrame):
	# conds = F
	most_watch = ["Child", "Movie", "Relax", "Sport", "TV"]
	columns = df.columns[1:]
	
	whens = F
	largest_dur = F.greatest(*columns)
	second_largest_dur = (
		F.when(col)
	)
	for column, name in zip(columns, most_watch):
		whens = whens.when(col(f"{column}") == largest_dur, lit(f"{name}"))
	whens = whens.otherwise("Error").alias("MostWatch")
	
	# conds = "when" + ".when".join(["(col('" + c + "') == col('LargestDuration'), lit('" + d + "'))" 
								   # for c, d in zip(columns, most_watch)])
	return (
		df
		.fillna(0)
		.select(
			"*",
			whens
		)
	)

In [11]:
def second_most_watch(
		df: DataFrame
):
	type_watch = ["Child", "Movie", "Relax", "Sport", "TV"]
	watch = ["MostWatch", "SecondMostWatch", "ThirdMostWatch"]
	columns = df.columns[1:]

	temp = F.sort_array(
		F.array(
			*[F.struct(col(c).alias('v'), F.lit(v).alias('k')
					   ) for c, v in zip(columns, type_watch)]
		),
		asc=False
	)
	return (
		df.select(
			df.Contract,
			df.TVDuration,
			df.ChildDuration,
			df.MovieDuration,
			df.RelaxDuration,
			df.SportDuration,
			temp[0]["k"].alias("MostWatch"),
			when(temp[1]["v"].isNotNull(), temp[1]["k"]).otherwise(None).alias("SecondMostWatch"),
			when(temp[2]["v"].isNotNull(), temp[2]["k"]).otherwise(None).alias("ThirdMostWatch"),
			# *[temp[i]["k"].alias(f"{x}") for i, x in zip([0, 1, 2], watch)]
		)
	)
	
	

In [12]:
second_most_watch(pivot_df).show()

+---------+----------+-------------+-------------+-------------+-------------+---------+---------------+--------------+
| Contract|TVDuration|ChildDuration|MovieDuration|RelaxDuration|SportDuration|MostWatch|SecondMostWatch|ThirdMostWatch|
+---------+----------+-------------+-------------+-------------+-------------+---------+---------------+--------------+
|VTH013504|    655434|         null|         null|         null|         null|       TV|           null|          null|
|HNJ121044|    484128|         null|        22629|         null|         null|       TV|          Movie|          null|
|GLD020852|   1517946|         1749|         null|         null|         null|       TV|          Child|          null|
|NDFD14513|   2881134|         null|         null|         null|         null|       TV|           null|          null|
|HPD082908|   1954119|        88605|         null|         null|         null|       TV|          Child|          null|
|CTFD19293|   2840349|         null|    

In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import count_distinct


def activeness(df: DataFrame):
	return (
		df.groupBy("Contract").agg(
			(count_distinct(col("Date"))/lit(30.)).cast(FloatType()).alias("Activeness")
		)
	)


In [ ]:
activeness(df).show()

In [ ]:
def pipe(self, func, *args, **kwargs):
	return func(self, *args, **kwargs)
DataFrame.pipe = pipe

In [ ]:
%load_ext memory_profiler

In [ ]:
def pipe_test(df):
	activeness_df = calculate_rfm(df)
	return \
		(df.pipe(pivot_data, app_names, column_names)
		 .pipe(second_most_watch)
		 .join(activeness_df, on="Contract")
		 )


In [ ]:
pipe_test(df).show()

In [ ]:
def normal_test(df):
	pivot_df = pivot_data(df, app_names, column_names)
	pivot_df = second_most_watch(pivot_df)
	activeness_df = activeness(df)
	return pivot_df.join(activeness_df, on="Contract")
	

In [ ]:
%memit normal_test(df)

In [7]:
from pyspark.sql.functions import max, array, array_max, datediff, min, round, ntile, count_distinct
from pyspark.sql import Window


def calculate_rfm(df: DataFrame):
	b = (
		df.groupBy("Contract").agg(
			max(col("Date")).alias("LatestDate"),
		)
	)

	rfm = (
		df
		.withColumn("ReportedDate", to_date(lit('20220501'), "yyyyMMdd"))
		.join(b, on="Contract")
		.groupBy(col("Contract"))
		.agg(
			min(datediff(col("ReportedDate"), col("LatestDate"))).alias("Recency"),
			round(count_distinct(col("Date")) / lit(30) * 100, 2).alias("Frequency"),
			sum(col("TotalDuration")).alias("Monetary_Duration")
		)
		.withColumns(
			{'Recency_score'          : ntile(4).over(Window.orderBy("Recency")),
			 'Frequency_score'        : ntile(4).over(Window.orderBy("Frequency")),
			 'Monetary_Duration_score': ntile(4).over(Window.orderBy("Monetary_Duration"))}
		)

	)
	return rfm


	


In [8]:
df = (
	spark.read.parquet(
		f"{data_path}{parquet_path}"
	)
	.filter(
		col("Date").between('2022-04-01', '2022-04-05')
	)
	.filter(
		(col("Contract") != "")
            & (col("Contract") != "0")
            & (col("TotalDuration") > 0)
	)
)

In [9]:
calculate_rfm(df).show()

23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 10:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 1

+---------+-------+---------+-----------------+-------------+---------------+-----------------------+
| Contract|Recency|Frequency|Monetary_Duration|Recency_score|Frequency_score|Monetary_Duration_score|
+---------+-------+---------+-----------------+-------------+---------------+-----------------------+
|AGAAA1218|     26|     3.33|               33|            1|              1|                      1|
|AGFD49198|     26|     3.33|               33|            1|              1|                      1|
|BEAAA0903|     26|     3.33|               33|            1|              1|                      1|
|BED006259|     26|     3.33|               33|            1|              1|                      1|
|BGFD58887|     26|     3.33|               33|            1|              1|                      1|
|BIFD07669|     26|     3.33|               33|            1|              1|                      1|
|BIFD22405|     26|     3.33|               33|            1|              1|     

In [51]:
import datetime
df.withColumn("ReportedDate", to_date(lit(datetime.datetime(2022, 5, 1)), "yyyyMMdd")).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `2022-05-01` cannot be resolved. Did you mean one of the following? [`AppName`, `Contract`, `Date`, `Id`, `Index`].;
'Project [Index#0, Type#1, Id#2, Score#3L, AppName#4, Contract#5, Mac#6, TotalDuration#7L, Date#8, to_date('2022-05-01, Some(yyyyMMdd), Some(Asia/Ho_Chi_Minh)) AS ReportedDate#1528]
+- Filter ((Date#8 >= cast(2022-04-01 as date)) AND (Date#8 <= cast(2022-04-05 as date)))
   +- Relation [Index#0,Type#1,Id#2,Score#3L,AppName#4,Contract#5,Mac#6,TotalDuration#7L,Date#8] parquet


In [55]:
df.createOrReplaceTempView("log_content")

## DeltaLake 

In [21]:
table_manager = TableManager(spark)

In [20]:
df = (
		spark.read.json(f"{data_path}/log_content/*.json")
		.select(
			col("_index").alias("Index"),
			col("_type").alias("Type"),
			col("_id").alias("Id"),
			col("_score").alias("Score"),
			col("_source.*"),
			to_date(
				regexp_extract(input_file_name(), r"\d{8}", 0),
				"yyyyMMdd"
			).alias("Date")
		)
	.filter(col("Date").between('2022-04-01', '2022-04-05'))
)
df.printSchema()

root
 |-- Index: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- AppName: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Mac: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Date: date (nullable = true)


In [24]:
raw_table_schema = [
	("Index", "STRING"),
	("Type", "STRING"),
	("Id", "STRING"),
	("Score", "LONG"),
	("AppName", "STRING"),
	("Contract", "STRING"),
	("MAC", "STRING"),
	("TotalDuration", "LONG"),
	("Date", "DATE"),
]
raw_table_path = "/Users/sonle/Documents/GitHub/spark-playground/data/DeltaFolder/raw_table"

### Create, Load and Delete Table

In [25]:
table_manager.create_table("raw_table", raw_table_schema, raw_table_path, is_partition=True, partition_by="Date")

In [39]:
spark.sql("describe delta.`{}`".format("/Users/sonle/Documents/GitHub/spark-playground/data/delta/silver/rfm")).show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            Contract|   string|   null|
|             Recency|      int|   null|
|           Frequency|   double|   null|
|   Monetary_Duration|   bigint|   null|
|       Recency_Score|      int|   null|
|     Frequency_Score|      int|   null|
|Monetary_Duration...|      int|   null|
+--------------------+---------+-------+


In [27]:
table_manager.load_table(df, raw_table_path)

In [34]:
spark.sql("describe detail '{}`".format("/Users/sonle/Documents/GitHub/spark-playground/data/delta/bronze")).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '''.(line 1, pos 16)

== SQL ==
describe detail '/Users/sonle/Documents/GitHub/spark-playground/data/delta/bronze`
----------------^^^


In [21]:
spark.sql("drop table if exists raw_table").show()

++
||
++
++


In [25]:
spark.sql("select * from raw_table").show()

+-----+----+---+-----+-------+--------+---+-------------+----+
|Index|Type| Id|Score|AppName|Contract|MAC|TotalDuration|Date|
+-----+----+---+-----+-------+--------+---+-------------+----+
+-----+----+---+-----+-------+--------+---+-------------+----+


In [31]:
delta_df = spark.read.format("delta").load(f"{raw_table_path}")

In [32]:
%%time 
pivot_delta_df = pivot_data(delta_df, app_names=app_names, column_names=column_names)

CPU times: user 10.9 ms, sys: 3.96 ms, total: 14.8 ms
Wall time: 2.89 s


In [30]:
%%time
pivot_data(df, app_names=app_names, column_names=column_names)

CPU times: user 14.5 ms, sys: 6.44 ms, total: 21 ms
Wall time: 2.64 s


DataFrame[Contract: string, ChildDuration: bigint, MovieDuration: bigint, RelaxDuration: bigint, SportDuration: bigint, TVDuration: bigint]

In [24]:
from delta.tables import  DeltaTable

deltaTable = DeltaTable.forPath(spark, raw_table_path)
deltaTable.delete()

In [11]:
deltaTable.history()

DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

In [12]:
delta_df = (
	spark.read.format("delta").load(f"{data_path}{parquet_path}")
)


In [14]:
parquet_df = df = (
	spark.read.parquet(
		f"{data_path}{parquet_path}"
	)
)

In [15]:
%%time
pivot_data(parquet_df, app_names=app_names, column_names=column_names)

CPU times: user 23.5 ms, sys: 8.88 ms, total: 32.4 ms
Wall time: 10.8 s


DataFrame[Contract: string, ChildDuration: bigint, MovieDuration: bigint, RelaxDuration: bigint, SportDuration: bigint, TVDuration: bigint]

In [46]:
deltaTable.optimize().executeCompaction()

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [52]:
deltaTable.optimize().executeZOrderBy("Type")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

## Validation with Pandera

In [27]:
df = (
	spark.read.parquet(
		f"{data_path}{parquet_path}"
	)
	.filter(
		col("Date").between('2022-04-01', '2022-04-12')
	)
)
df.printSchema()

root
 |-- Index: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- AppName: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Mac: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Date: date (nullable = true)


In [36]:
pivot_df = pivot_data(df, app_names=app_names, column_names=column_names)

In [6]:
from pyspark.sql.types import LongType
import pandera.pyspark as pa

from pandera.pyspark import DataFrameModel

class PivotSchema(DataFrameModel):
	Contract: StringType = pa.Field(nullable=True)
	MovieDuration: LongType = pa.Field(nullable=True)
	ChildDuration: LongType = pa.Field(nullable=True)
	RelaxDuration: LongType = pa.Field(nullable=True)
	SportDuration: LongType = pa.Field(nullable=True)
	TVDuration: LongType = pa.Field(nullable=True)


/Users/sonle/Library/Caches/pypoetry/virtualenvs/spark-playground-WomFRekb-py3.11/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [35]:
%env PANDERA_CACHE_DATAFRAME=False

env: PANDERA_CACHE_DATAFRAME=False


In [37]:
val_df_false = PivotSchema.validate(pivot_df)

In [38]:
%env PANDERA_CACHE_DATAFRAME=True

env: PANDERA_CACHE_DATAFRAME=True


In [39]:
val_df_true = PivotSchema.validate(pivot_df)

In [34]:
error = val_df.pandera.errors
if error is None:
	print(error)

In [21]:
validated_schema = {
	"pivot" : PivotSchema
}

In [22]:
validated_schema.get("pivot").validate(pivot_df)

PivotSchema

In [19]:
publish = False
if publish:
	print("Publishing")